In [8]:
import asyncio
import json
import os
import nest_asyncio
import pprint 
import base64
from io import BytesIO
from playwright.async_api import async_playwright
import google.generativeai as genai
#import tubulate
from PIL import Image
from IPython.display import display, HTML,Markdown
from pydantic import BaseModel
from helper import get_openai_api_key,visualizeCourses,get_openai_client

In [10]:
client = get_openai_client()
nest_asyncio.apply()

response = client.generate_content("Explain AI in simple terms.")
print(response.text)

Imagine you have a really smart dog.  You teach it tricks by showing it examples and rewarding good behavior.  AI is kind of like that, but instead of a dog, it's a computer program.

We "teach" the computer program by showing it lots of examples – like pictures of cats and dogs to teach it the difference.  The program then learns patterns and rules from these examples so it can identify new pictures of cats and dogs it's never seen before.

AI can do much more than just identify pictures. It can:

* **Understand language:**  Like Siri or Alexa understanding your voice commands.
* **Play games:**  Like chess or Go, sometimes even better than humans.
* **Recommend things:** Like movies or products you might like based on your past choices.
* **Drive cars:**  Self-driving cars use AI to navigate roads and avoid obstacles.

Basically, AI is about making computers smarter and able to do things that usually require human intelligence.  It's not actually "thinking" like a human, but it can s

WebScraper Agent

In [12]:
class WebScraperAgent:
    def __init__(self):
        self.playwright = None
        self.browser = None
        self.page = None

    async def init_browser(self):
        self.playwright = await async_playwright().start()
        self.browser = await self.playwright.chromium.launch(
            headless=True,
            args=[
                "--disable-dev-shm-usage",
                "--no-sandbox",
                "--disable-setuid-sandbox",
                "--disable-accelerated-2d-canvas",
                "--disable-gpu",
                "--no-zygote",
                "--disable-audio-output",
                "--disable-software-rasterizer",
                "--disable-webgl",
                "--disable-web-security",
                "--disable-features=LazyFrameLoading",
                "--disable-features=IsolateOrigins",
                "--disable-background-networking"

            ]
        )
        self.page = await self.browser.new_page()

    async def scrape_content(self, url):
        if not self.page or self.page.is_closed():
            await self.init_browser()
        await self.page.goto(url,wait_untill="load")
        await self.page.wait_for_timeout(2000)
        return await self.page.content()
    
    async def take_screenshot(self, path="screenshot.png"):
        await self.page.screenshot(path=path,full_page=True)
        return path
    
    async def screenshot_buffer(self):
        screenshot_bytes=await self.page.screenshot(type="png",full_page=False)
        return screenshot_bytes
    
    async def close(self ):
        await self.browser.close()
        await self.playwright.stop()
        self.playwright=None
        self.browser=None
        self.page=None

